In [2]:
!pip install --no-index --find-links="/kaggle/input/surface-package-scraper" -q pytorch_lightning monai albumentations imagecodecs --no-deps # "numpy==1.26.4" "scipy==1.15.3"
!pip uninstall -q -y tensorflow  # preventing AttributeError

^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user


In [3]:
# Cell 1
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

1 - 384


40 - 256

750 - 320

In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
import gc
import warnings
from typing import Tuple, Optional, Dict, List, Callable
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import os
warnings.filterwarnings("ignore")

images_path = "/kaggle/input/vesuvius-surface-npz/train_images"
mask_path = "/kaggle/input/vesuvius-surface-npz/train_labels"
root_dir = "/kaggle/input/vesuvius-surface-npz"
test_images_path = "/kaggle/input/vesuvius-challenge-surface-detection/test_images"
OUTPUT_DIR = "/kaggle/working/checkpoints"
os.makedirs(OUTPUT_DIR, exist_ok=True)
gc.collect()

ModuleNotFoundError: No module named 'version'

In [ ]:
class Config:
    def __init__(self):
        self.lr = 1e-4
        self.num_workers = 2
        self.batches = 1
        self.val_split = 0.2
        self.target_shape = (128,128,128)
        self.weight_decay = 2e-4
        self.max_epochs = 50
        

config = Config()

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np
from pathlib import Path
import tifffile
import pytorch_lightning as pl
import random
from monai import transforms as MT
class VesuviusDataset(Dataset):
    def __init__(self, 
                 images_id: list, 
                 images_dir: str = images_path, 
                 test_dir: str = test_images_path,   
                 masks_dir: str = mask_path, 
                 mode: str = "train"):
        
        self.images_id = images_id
        self.mode = mode
        self.images_dir = Path(images_dir)
        self.masks_dir = Path(masks_dir) if masks_dir else None
        self.test_dir = Path(test_dir) if test_dir else None

    def __len__(self):
        return len(self.images_id)

    def __getitem__(self, idx):
        file_name = self.images_id[idx]
        
        # 1. Determine Source Directory
        if self.mode == "test":
            if self.test_dir is None:
                raise ValueError("Mode is 'test' but no 'test_dir' was provided!")
            source_dir = self.test_dir
        else:
            # Both 'train' and 'val' come from the training images directory
            source_dir = self.images_dir

        # 2. Load Image
        img_path = source_dir / file_name
        img = self.load_volume(img_path)

        # 3. Load Mask (For both Train AND Val)
        if self.mode in ["train", "val"]:
            if self.masks_dir is None:
                raise ValueError(f"Mode is '{self.mode}' but no 'masks_dir' was provided!")
            
            mask_path = self.masks_dir / file_name
            mask = self.load_volume(mask_path, is_mask=True)
            return img, mask, file_name
            
        # 4. Test mode (No mask)
        else:
            return img, None, file_name

    def load_volume(self, file_path, is_mask=False):
        path_obj = Path(file_path)
        if not path_obj.exists():
            raise FileNotFoundError(f"Could not find file: {path_obj}")
        if path_obj.suffix == ".npz":
            archive = np.load(str(path_obj))
            data = archive[list(archive.files)[0]]
        elif path_obj.suffix in [".tif", ".tiff"]:
            data = tifffile.imread(str(path_obj))
        else:
            data = np.load(str(path_obj))
        tensor = torch.from_numpy(data)

        if not is_mask:
            tensor = tensor.half().div_(255.0).unsqueeze(0)
        else:
            tensor = tensor.long().unsqueeze(0)

        return tensor

In [ ]:
import os
images_id = None
for _,_,c in os.walk(images_path):
    images_id = c


In [ ]:
def custom_collate(batch):
    return batch
#we want colation on happen on gpu as cpu cant do heavy interpolation
#skipping the collation in dataloader

In [ ]:
import pytorch_lightning 
class DesuviusDataModule(pl.LightningDataModule):

    def __init__(self,
                 train_img_dir = images_path,
                 test_img_dir = test_images_path,
                 mask_img_dir = mask_path,
                 num_workers = config.num_workers,
                 batches = config.batches,
                 target_shape = config.target_shape):
        super().__init__()
        self.train_img_dir = train_img_dir
        self.test_img_dir= test_img_dir
        self.mask_img_dir = mask_img_dir
        self.num_workers = num_workers
        self.batches = batches
        self.target_shape = target_shape
        self.val_split = config.val_split

        self.train_dataset = None
        self.test_dataset = None
        self.mask_dataset = None

        self.gpu_augments = MT.Compose([
            MT.Resized(keys=["image", "label"], spatial_size=self.target_shape, mode=["trilinear", "nearest"]),
            MT.RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
            MT.RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=1),
            MT.RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=2),
            MT.RandRotated(keys=["image", "label"], range_x=0.1, range_y=0.1, range_z=0.1, prob=0.3, keep_size=True, mode=["bilinear", "nearest"]),
            MT.RandShiftIntensityd(keys=["image"], offsets=0.1, prob=0.5),
            MT.RandGaussianNoised(keys=["image"], prob=0.3, mean=0.0, std=0.01),
        ])
        # Validation transforms: Just Resize (for image AND label)
        self.val_augments = MT.Compose([
            MT.Resized(keys=["image", "label"], spatial_size=self.target_shape, mode=["trilinear", "nearest"])
        ])
        # Validation transforms for Image ONLY (for test set where labels are None)
        self.val_image_augments = MT.Compose([
            MT.Resized(keys=["image"], spatial_size=self.target_shape, mode=["trilinear"])
        ])

    def setup(self, stage = None):
        random.seed(32)
        all_files = images_id # Assumes images_id is defined globally
        random.shuffle(all_files)
        # Split into train/val
        split_idx = int(len(all_files) * (1 - self.val_split))
        train_files = all_files[:split_idx]
        val_files = all_files[split_idx:]
        print(f"Total files: {len(all_files)}")
        print(f"Train files: {len(train_files)}")
        print(f"Val files: {len(val_files)}")

        # FIX: Use the split files, not the global list
        self.train_dataset = VesuviusDataset(images_id = train_files, mode = "train")
        self.val_dataset = VesuviusDataset(images_id = val_files, mode = "val")

    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            self.train_dataset,
            batch_size = self.batches,
            shuffle = True,
            num_workers = self.num_workers,
            pin_memory = True,
            persistent_workers=bool(self.num_workers > 0),
            collate_fn = custom_collate
        )
    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            self.val_dataset,
            batch_size = self.batches,
            shuffle = False, # FIX: Should be False for validation
            num_workers = self.num_workers,
            pin_memory = True,
            persistent_workers=bool(self.num_workers > 0),
            collate_fn = custom_collate
        )

    def on_after_batch_transfer(self, batch, dataloader_idx): 
        if not isinstance(batch, list):
            return super().on_after_batch_transfer(batch, dataloader_idx)

        x_list, y_list, frag_ids = [], [], [] # FIX: renamed z_list to frag_ids
        
        # Determine device
        device = self.trainer.strategy.root_device if self.trainer else torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for item in batch:
            x, y, frag_id = item
            x = x.to(device, non_blocking=True)
            
            # Logic: If we are training, use gpu_augments. 
            # If validating and y exists, use val_augments.
            # If y is None (predict), use val_image_augments.
            
            if y is not None:
                y = y.to(device, non_blocking=True)
                data = {"image": x, "label": y}
                
                # Pick transform
                if self.trainer.training:
                    trans = self.gpu_augments
                else:
                    trans = self.val_augments
                
                data = trans(data) # FIX: Use selected transform
                y_list.append(data["label"])
            else:
                # Prediction case (No Label)
                data = {"image": x}
                trans = self.val_image_augments
                data = trans(data)
                # Do not append to y_list
            
            x_list.append(data["image"])
            frag_ids.append(frag_id)

        # Handle Y stack (if empty)
        y_stack = torch.stack(y_list) if len(y_list) > 0 else None
        
        return torch.stack(x_list), y_stack, frag_ids # FIX: Return list frag_ids

In [ ]:
dataModule = DesuviusDataModule()
dataModule.setup()

In [ ]:
print(dataModule.train_dataloader)

In [ ]:
import matplotlib.pyplot as plt

# Get a batch
train_loader = dataModule.train_dataloader()
batch = next(iter(train_loader))

# With custom_collate, batch is a list: [(img, mask, id), ...]
# We extract the first sample
raw_img, raw_mask, frag_id = batch[0]

print(f"Raw ID: {frag_id}")
print(f"Raw Image Shape: {raw_img.shape}")

# Manually apply the validation transform (Resize) to visualize the model input
# We use val_augments which only contains Resized
# Construct dictionary as expected by MONAI transforms
data = {"image": raw_img, "label": raw_mask}
data_resized = dataModule.val_augments(data)

images = data_resized["image"]
masks = data_resized["label"]

# Select first channel
img = images[0].numpy()  # (D, H, W)
msk = masks[0].numpy()   # (D, H, W)

print(f"Resized Image Shape: {img.shape}")
print(f"Mask Shape: {msk.shape}")
print(f"Image Range: {img.min()} - {img.max()}")

# Calculate middle indices
d_mid = img.shape[0] // 2
h_mid = img.shape[1] // 2
w_mid = img.shape[2] // 2

# Setup plot: 3 Rows (Axes), 2 Cols (Image, Mask)
fig, axes = plt.subplots(3, 2, figsize=(10, 15))

# Row 1: Z-axis (Depth/Axial)
axes[0, 0].imshow(img[d_mid, :, :], cmap='gray')
axes[0, 0].set_title(f'Axial (Depth={d_mid}) - Image')
axes[0, 1].imshow(msk[d_mid, :, :], cmap='gray')
axes[0, 1].set_title(f'Axial (Depth={d_mid}) - Mask')

# Row 2: Y-axis (Height/Coronal)
axes[1, 0].imshow(img[:, h_mid, :], cmap='gray')
axes[1, 0].set_title(f'Coronal (Height={h_mid}) - Image')
axes[1, 1].imshow(msk[:, h_mid, :], cmap='gray')
axes[1, 1].set_title(f'Coronal (Height={h_mid}) - Mask')

# Row 3: X-axis (Width/Sagittal)
axes[2, 0].imshow(img[:, :, w_mid], cmap='gray')
axes[2, 0].set_title(f'Sagittal (Width={w_mid}) - Image')
axes[2, 1].imshow(msk[:, :, w_mid], cmap='gray')
axes[2, 1].set_title(f'Sagittal (Width={w_mid}) - Mask')

plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from monai.losses import TverskyLoss

class surfaceSegementation(pl.LightningModule):
    def __init__(self, net, learning_rate=1e-3, weight_decay=1e-4):
        super().__init__()
        # Fixed typo: save_hyperparameters
        self.save_hyperparameters(ignore=["net"])
        # Fixed: assigned 'net' instead of undefined 'model'
        self.net_module = net
        # Fixed: removed undefined 'config' object and used init args
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.out_channels = 2
        self.spatial_dims  = 3
        self.ignore_index_val = 2
        
        self.crossEntropy = nn.CrossEntropyLoss(
            ignore_index = self.ignore_index_val
        )
        self.Tversky = TverskyLoss(
            softmax = True,
            to_onehot_y = False,
            include_background = True,
            alpha = 0.75,
            beta = 0.25  
        )

    # Added missing forward method required for self(img)
    def forward(self, x):
        return self.net_module(x)

    def _compute_loss(self, logits, targets):
        # pytorch cross entropy can ignore classes inherintly
        targets = targets.squeeze(1)
        ce_loss = self.crossEntropy(logits, targets.long())
        
        # mask so tversky doesnt get confused by class 2 as it cant ignore it natively
        # Fixed: Changed 'target' to 'targets'
        mask = (targets != self.ignore_index_val)
        
        # setting index 2 to 0
        # Fixed: Changed 'target' to 'targets'
        target_clean = torch.where(mask, targets, torch.tensor(0, device=targets.device))
        
        targets_onehot = torch.nn.functional.one_hot(
            target_clean.long(),
            num_classes=2
        ).float()

        # Fixed: Changed self.hparams.spatial_dims to self.spatial_dims
        if self.spatial_dims == 3:
            targets_onehot = targets_onehot.permute(0,4,1,2,3)
        else:
            targets_onehot = targets_onehot.permute(0,3,1,2) # Fixed permute for 2D
        
        # CHANGED: .half() -> .float() to prevent runtime errors on standard FP32 training
        targets_masked_ohe = targets_onehot * mask.unsqueeze(1).float() 

        tversky_loss = self.Tversky(logits, targets_masked_ohe)

        return 0.5*tversky_loss + 0.5*ce_loss

    def _compute_metrics(self, preds_logits: torch.Tensor, targets_class_indices: torch.Tensor) -> dict:
        """
        Computes Dice and IoU fully vectorized (No for-loops).
        """
        num_classes = preds_logits.shape[1]
        preds_hard = torch.argmax(preds_logits, dim=1, keepdim=True)
        valid_mask = (targets_class_indices != self.ignore_index_val)
        
        # 3. One-Hot Encode Predictions
        preds_ohe = torch.nn.functional.one_hot(
            preds_hard.squeeze(1), 
            num_classes=num_classes
        ).float()
        
        # 4. One-Hot Encode Targets
        targets_clean = torch.where(valid_mask, targets_class_indices, torch.tensor(0, device=targets_class_indices.device))
        
        targets_ohe = torch.nn.functional.one_hot(
            targets_clean.squeeze(1).long(), 
            num_classes=num_classes
        ).float()

        # Fixed: Changed self.hparams.spatial_dims to self.spatial_dims
        if self.spatial_dims == 3:
            preds_ohe = preds_ohe.permute(0, 4, 1, 2, 3)
            targets_ohe = targets_ohe.permute(0, 4, 1, 2, 3)
        else:
            preds_ohe = preds_ohe.permute(0, 3, 1, 2)
            targets_ohe = targets_ohe.permute(0, 3, 1, 2)

        valid_mask_float = valid_mask.float()
        preds_ohe = preds_ohe * valid_mask_float
        targets_ohe = targets_ohe * valid_mask_float

        # Fixed: Changed self.hparams.spatial_dims to self.spatial_dims
        reduce_dims = (0, 2, 3, 4) if self.spatial_dims == 3 else (0, 2, 3)

        intersection = (preds_ohe * targets_ohe).sum(dim=reduce_dims)
        cardinality_pred = preds_ohe.sum(dim=reduce_dims)
        cardinality_target = targets_ohe.sum(dim=reduce_dims)
        
        union_sum = cardinality_pred + cardinality_target

        dice_scores = (2. * intersection + 1e-8) / (union_sum + 1e-8)
        iou_scores = (intersection + 1e-8) / (union_sum - intersection + 1e-8)

        return {
            "dice": dice_scores.mean(),
            "iou": iou_scores.mean()
        }

    # Added 'self' argument
    def training_step(self, batch, batch_idx):
        img, mask, _ = batch
        logits = self(img)
        loss = self._compute_loss(logits, mask)
        # Fixed function name _compute_metric -> _compute_metrics
        # Fixed variable name metric -> metrics (to match logging below)
        metrics = self._compute_metrics(logits, mask)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("train_dice", metrics["dice"], on_step=True, on_epoch=True, prog_bar=True)
        self.log("train_iou", metrics["iou"], on_step=True, on_epoch=True, prog_bar=True)
        return loss

    # Added 'self' argument
    def validation_step(self, batch, batch_idx):
        img, mask, _ = batch
        logits = self(img)
        loss = self._compute_loss(logits, mask)
        # Fixed function name _compute_metric -> _compute_metrics
        # Fixed variable name metric -> metrics
        metrics = self._compute_metrics(logits, mask)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_dice", metrics["dice"], on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_iou", metrics["iou"], on_step=False, on_epoch=True, prog_bar=True)
        return loss

    # Added 'self' argument and renamed to predict_step
    def predict_step(self, batch, batch_idx):
        img, _, frag_id = batch
        logits = self(img)
        prob = torch.softmax(logits, dim=1)
        classes = torch.argmax(prob, dim=1)
        return {"prediction": classes, "frag_id": frag_id}
        
    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            # Fixed MAX_EPOCHS to use self.trainer.max_epochs
            T_max=self.trainer.max_epochs if self.trainer else 100,
            eta_min=1e-6
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "epoch"
            }
        }

In [ ]:
from monai.networks.nets import SegResNet, SwinUNETR
net = SwinUNETR(
    in_channels=1,
    out_channels=2,
    feature_size=48,
    use_v2=True,
    drop_rate=0.2,
    attn_drop_rate=0.2,
    dropout_path_rate=0.2,
    use_checkpoint=True
)

net_name = net.__class__.__name__
model = surfaceSegementation(net=net)

In [ ]:
ckpt_path = "/kaggle/input/vesuvius-swinuneter-model1/checkpoints/SwinUNETR-epoch=29-val_dice=0.7091.ckpt"

In [ ]:
# import pytorch_lightning as pl
# from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
# from pytorch_lightning.loggers import CSVLogger
# from pytorch_lightning.utilities.exceptions import MisconfigurationException

# # Callbacks
# checkpoint_callback = ModelCheckpoint(
#     dirpath=OUTPUT_DIR,
#     filename=net_name + "-{epoch:02d}-{val_dice:.4f}",
#     monitor="val_dice",
#     mode="max",
#     save_top_k=3,
#     verbose=True
# )

# early_stop_callback = EarlyStopping(
#     monitor="val_dice",
#     patience=10,
#     mode="max",
#     verbose=True
# )

# lr_monitor = LearningRateMonitor(logging_interval="epoch")
# csv_logger = CSVLogger(save_dir=OUTPUT_DIR)

# # Trainer
# trainer = pl.Trainer(
#     max_epochs=config.max_epochs,
#     accelerator="auto",
#     devices="auto",
#     logger=csv_logger,
#     callbacks=[checkpoint_callback, early_stop_callback, lr_monitor],
#     precision="16-mixed",
#     log_every_n_steps=20,
#     enable_progress_bar=True,
#     accumulate_grad_batches=18,
#     gradient_clip_val=1.0, # Clips gradient norm to 1.0 to prevent exploding gradients
# )

# # Train
# try:
#     trainer.fit(model, datamodule=dataModule, ckpt_path=ckpt_path)
# except MisconfigurationException as ex:
#     print(ex)

In [ ]:
import pandas as pd
metrics1 = pd.read_csv("/kaggle/input/vesuvius-swinuneter-model1/checkpoints/lightning_logs/version_0/metrics.csv")
metrics1.head()
metrics1['epoch'].unique()


In [ ]:
metrics2 = pd.read_csv("/kaggle/input/vesuvius-baseline-model/checkpoints/lightning_logs/version_0/metrics.csv")
metrics2.head()
metrics2['epoch'].unique()

In [ ]:
metrics1.head()

In [ ]:
import pandas as pd

# 1. Load the two CSV files
# Replace 'metrics_0_30.csv' and 'metrics_30_50.csv' with your actual file names
df1 = pd.read_csv('/kaggle/input/vesuvius-swinuneter-model1/checkpoints/lightning_logs/version_0/metrics.csv')
df2 = pd.read_csv('/kaggle/input/vesuvius-baseline-model/checkpoints/lightning_logs/version_0/metrics.csv')

# 2. Concatenate them
# ignore_index=True is important here: it resets the index so it runs continuously
# from 0 to the end, instead of restarting at 0 for the second file.
merged_df = pd.concat([df1, df2], ignore_index=True)

# 3. Check the result
print(f"Shape of first file: {df1.shape}")
print(f"Shape of second file: {df2.shape}")
print(f"Shape of merged file: {merged_df.shape}")

# 4. Save the merged data to a new CSV
merged_df.to_csv('merged_metrics.csv', index=False)

In [ ]:
merged_df.columns


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming your dataframe is named 'df'

# 1. Clean the data by grouping by epoch
# This merges rows so that for each epoch, you have the values for all metrics
# .mean() works well here because the non-NaN values are usually unique per epoch
epoch_df = merged_df.groupby('epoch').mean()

# Reset index so 'epoch' becomes a column again for plotting
epoch_df = epoch_df.reset_index()

# 2. Setup the plots
sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# --- Plot 1: Dice Scores ---
# using 'train_dice_epoch' vs 'val_dice'
axes[0].plot(epoch_df['epoch'], epoch_df['train_dice_epoch'], label='Train Dice', marker='o')
axes[0].plot(epoch_df['epoch'], epoch_df['val_dice'], label='Val Dice', marker='o')
axes[0].set_title('Dice Score vs Epoch', fontsize=14)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Dice Score', fontsize=12)
axes[0].legend()

# --- Plot 2: Loss ---
# using 'train_loss_epoch' vs 'val_loss'
axes[1].plot(epoch_df['epoch'], epoch_df['train_loss_epoch'], label='Train Loss', marker='o')
axes[1].plot(epoch_df['epoch'], epoch_df['val_loss'], label='Val Loss', marker='o')
axes[1].set_title('Loss vs Epoch', fontsize=14)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].legend()

# --- Plot 3: IoU ---
# using 'train_iou_epoch' vs 'val_iou'
axes[2].plot(epoch_df['epoch'], epoch_df['train_iou_epoch'], label='Train IoU', marker='o')
axes[2].plot(epoch_df['epoch'], epoch_df['val_iou'], label='Val IoU', marker='o')
axes[2].set_title('IoU vs Epoch', fontsize=14)
axes[2].set_xlabel('Epoch', fontsize=12)
axes[2].set_ylabel('IoU', fontsize=12)
axes[2].legend()

plt.tight_layout()
plt.show()

In [ ]:
checkpoint_path = "/kaggle/input/vesuvius-baseline-model/checkpoints/SwinUNETR-epoch=47-val_dice=0.7371.ckpt"

In [ ]:

model = surfaceSegementation.load_from_checkpoint(
    checkpoint_path, 
    net=net
)

# 3. Prepare for Inference
model.eval()        # Disables dropout/batchnorm updates
model.freeze()      # Sets requires_grad=False to save memory
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print("done")